In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:

import matplotlib.pyplot as plt
import seaborn as sns
# # Hungary
# df = pd.read_csv('../input/heart-attack-prediction/data.csv')
# df.info()

# # Full
# df = pd.read_csv('../input/heart-disease-dataset/heart.csv')
# df.info()

# Cleavland
df  = pd.read_csv("../input/heart-disease-uci/heart.csv")
df.info()



# Data Exploration

In [ ]:
sns.countplot(x='target',data=df)
plt.show()

In [ ]:
plt.figure(figsize=(12,4))
sns.distplot(df['age'])

In [ ]:
plt.figure(figsize=(12,12))
sns.heatmap(df.corr(),annot=True,cmap='viridis')
plt.ylim(14, 0)
plt.xlim(0, 14)

In [ ]:
plt.figure(figsize=(8,8))
sns.scatterplot(x='age',y='thalach',data=df)

In [ ]:
plt.figure(figsize=(8,8))
sns.boxplot(x='target',y='age',data=df)

In [ ]:
df.groupby('target')['age'].describe()

In [ ]:
sorted(df['cp'].unique())
sns.countplot(x='cp',data=df,hue='target')

In [ ]:
plt.figure(figsize=(20,5))
cp_order = sorted(df['age'].unique())
sns.countplot(x='age',data=df,order = cp_order,palette='coolwarm' ,hue='cp')

In [ ]:
plt.figure(figsize=(20,5))
df.corr()['age'].sort_values().drop('age')

In [ ]:
df.head()

In [ ]:
len(df)

In [ ]:
df.isnull().sum()

In [ ]:
age_1 = df[df['target']==1].groupby("age").count()['target']
age_0 = df[df['target']==0].groupby("age").count()['target']


In [ ]:
 (age_1/(age_1 + age_0))

In [ ]:
plt.figure(figsize=(17,8))
age_len = age_0/age_1
age_len.plot(kind = 'bar')

## Split Data

In [ ]:
from sklearn.model_selection import train_test_split
df=df.sample(frac=1)

In [ ]:

X = df.drop('target', axis =1).values
X

In [ ]:
y = df['target'].values
y

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=101)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation,Dropout
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from datetime import datetime

early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=30)

time = datetime.now().strftime("%Y-%m-%d--%H%M")
logdir = 'logs\\hparam_tuning'
# board = TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True, write_images = True, 
#                     update_freq='epoch', profile_batch=2, embeddings_freq=1)

In [ ]:
from tensorboard.plugins.hparams import api as hp
HP_NUM_UNITS_L1 = hp.HParam('num_units_l1', hp.Discrete([16, 32]))
HP_NUM_UNITS_L2 = hp.HParam('num_units_l2', hp.Discrete([8,4]))
HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.1, 0.2))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs\\hparam_tuning').as_default():
    hp.hparams_config(
        hparams=[HP_NUM_UNITS_L1,HP_NUM_UNITS_L2, HP_DROPOUT, HP_OPTIMIZER],
        metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )

def train_test_model(hparams):
    model = Sequential()
    # input layer
    model.add(Dense(hparams[HP_NUM_UNITS_L1],  activation='relu'))
    model.add(Dropout(hparams[HP_DROPOUT]))

    # hidden layer
    model.add(Dense(hparams[HP_NUM_UNITS_L2], activation='relu'))
    model.add(Dropout(hparams[HP_DROPOUT]))
    
    model.add(Dense(8, activation='relu'))
    model.add(Dropout(hparams[HP_DROPOUT]))
    # output layer
    model.add(Dense(units=1,activation='sigmoid'))

    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=hparams[HP_OPTIMIZER],  metrics=['accuracy'])
    
    model.fit(x=X_train, 
          y=y_train, 
          epochs=400,
          validation_data=(X_test, y_test), 
          verbose=1,
          callbacks = [early_stop,
                       tf.keras.callbacks.TensorBoard(logdir),  # log metrics
                        hp.KerasCallback(logdir, hparams)]
          )
    
    _, accuracy = model.evaluate(X_test, y_test)
    
    return accuracy

def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        accuracy = train_test_model(hparams)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

session_num = 0

for num_units_l1 in HP_NUM_UNITS_L1.domain.values:
    for num_units_l2 in HP_NUM_UNITS_L2.domain.values:
        for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
            for optimizer in HP_OPTIMIZER.domain.values:
              hparams = {
                  HP_NUM_UNITS_L1: num_units_l1,
                  HP_NUM_UNITS_L2: num_units_l2,
                  HP_DROPOUT: dropout_rate,
                  HP_OPTIMIZER: optimizer,
              }
              run_name = "run-%d" % session_num
              print('--- Starting trial: %s' % run_name)
              print({h.name: hparams[h] for h in hparams})
              run('logs\\hparam_tuning' + run_name, hparams)
              session_num += 1

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
predictions = model.predict_classes(X_test)
print(classification_report(y_test,predictions))

In [ ]:
confusion_matrix(y_test,predictions)

In [ ]:
row = 300
patient = df.drop('target', axis = 1).iloc[row]
patient['age'] = 20
patient['cp']=0.0
patient

In [ ]:
patient = scaler.transform(patient.values.reshape(1,13))
patient

In [ ]:
model.predict_classes(patient)[0]

In [ ]:
df.iloc[row]['target']

In [ ]:
result = []
for row in range (0,300):
    patient = df.drop('target', axis = 1).iloc[row]
    patient = scaler.transform(patient.values.reshape(1,13))
    result = result + model.predict_classes(patient) == df.iloc[row]['target']
print(result)